# Life Expectancy- Modelling & Deployment

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pd.pandas.set_option('display.max_columns',None)

In [10]:
#importing the feature engineered dataset New.csv

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ae1b71cc049e4c9b85a0caaac9f48f6b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Paa-MBSOSgWmzwPOzLK5V-axW52wUerKWyGvIZzOYRWz',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_ae1b71cc049e4c9b85a0caaac9f48f6b.get_object(Bucket='firstproject-donotdelete-pr-pkwev7v7emc99n',Key='New.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,31,2015,0,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,31,2014,0,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,31,2013,0,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,31,2012,0,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,31,2011,0,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [11]:
x = df.drop(['Life expectancy '],axis=1)
y = df['Life expectancy ']
type(x)

pandas.core.frame.DataFrame

In [12]:
#splitting the dataset for Training and Testing
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

## Apllying Random Forest Regressor

In [13]:
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor(n_estimators=10,random_state=0)
from sklearn.model_selection import cross_val_score
regressor.fit(X_train,y_train)
train_score = cross_val_score(regressor,X_train,y_train,cv=5)
test_score = cross_val_score(regressor,X_test,y_test,cv=5)

print("train score :",np.mean(train_score))
print("test score :",np.mean(test_score))

train score : 0.9681152721877062
test score : 0.9385265703438339


### Importing Watson Machine Learning API Client

In [14]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-06-14 18:11:07,339 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [15]:
wml_credentials = {
  "apikey": "rA_JS3oGch15W7TGG97h862gCdQz9TAggCu6pPXARFgx",
  "iam_apikey_description": "Auto-generated for key f426446b-d6e2-4191-97ce-db28a1c52340",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/f6ed1b7e1a784cfe883690a92e3f8b89::serviceid:ServiceId-4cc25370-25f3-454c-9710-f8cff0dfb754",
  "instance_id": "8f159afd-0624-46ce-bda0-e2e2f814b0e6",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

### Creating a client with our ML service credentials

In [16]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [17]:
model_props = {
    client.repository.ModelMetaNames.AUTHOR_NAME : "Martin",
    client.repository.ModelMetaNames.AUTHOR_EMAIL : "martinpathadan234@gmail.com",
    client.repository.ModelMetaNames.NAME : "Life_exp"
}

In [18]:
#Creating Watson Machine Learning Model
model_artifact = client.repository.store_model(regressor,meta_props = model_props)

In [19]:
model_artifact

{'metadata': {'guid': '6794e523-5926-4c64-a5e7-1370fb046429',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/8f159afd-0624-46ce-bda0-e2e2f814b0e6/published_models/6794e523-5926-4c64-a5e7-1370fb046429',
  'created_at': '2020-06-14T18:11:07.798Z',
  'modified_at': '2020-06-14T18:11:07.852Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'learning_configuration_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/8f159afd-0624-46ce-bda0-e2e2f814b0e6/published_models/6794e523-5926-4c64-a5e7-1370fb046429/learning_configuration',
  'author': {'name': 'Martin'},
  'name': 'Life_exp',
  'learning_iterations_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/8f159afd-0624-46ce-bda0-e2e2f814b0e6/published_models/6794e523-5926-4c64-a5e7-1370fb046429/learning_iterations',
  'feedback_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/8f159afd-0624-46ce-bda0-e2e2f814b0e6/published_models/6794e523-5926-4c64-a5e7-1370fb046429/feedback',
  'latest_version': {'url': 'https://

In [20]:
guid = client.repository.get_model_uid(model_artifact)
guid

'6794e523-5926-4c64-a5e7-1370fb046429'

### Deployment Creation

In [21]:
#Creating Deployment for our ML Model 
deploy = client.deployments.create(guid,name = "Life_exp_d")



#######################################################################################

Synchronous deployment creation for uid: '6794e523-5926-4c64-a5e7-1370fb046429' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='87e2501b-8386-4b28-b7c3-02b9fa191cef'
------------------------------------------------------------------------------------------------




In [22]:
#List of deployments
client.deployments.list()

------------------------------------  ----------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME        TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
87e2501b-8386-4b28-b7c3-02b9fa191cef  Life_exp_d  online  DEPLOY_SUCCESS  2020-06-14T18:11:10.507Z  scikit-learn-0.20  model
------------------------------------  ----------  ------  --------------  ------------------------  -----------------  -------------


In [24]:
deploy

{'metadata': {'guid': '87e2501b-8386-4b28-b7c3-02b9fa191cef',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/8f159afd-0624-46ce-bda0-e2e2f814b0e6/deployments/87e2501b-8386-4b28-b7c3-02b9fa191cef',
  'created_at': '2020-06-14T18:11:10.507Z',
  'modified_at': '2020-06-14T18:11:10.763Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'name': 'Life_exp_d',
  'scoring_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/8f159afd-0624-46ce-bda0-e2e2f814b0e6/deployments/87e2501b-8386-4b28-b7c3-02b9fa191cef/online',
  'deployable_asset': {'name': 'Life_exp',
   'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/8f159afd-0624-46ce-bda0-e2e2f814b0e6/published_models/6794e523-5926-4c64-a5e7-1370fb046429',
   'guid': '6794e523-5926-4c64-a5e7-1370fb046429',
   'created_at': '2020-06-14T18:11:10.483Z',
   'type': 'model'},
  'description': 'Description of deployment',
  'status_details': {'status': 'DEPLOY_SUCCESS'},
  'model_type': 'scikit-learn-0.20',
  'status': 'DEPLOY_SUC

### Scoring endpoint url

In [1]:
#The scoring endpoint url can be used to create ui for our deployment

In [25]:
scoring_url = client.deployments.get_scoring_url(deploy)

In [26]:
scoring_url

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/8f159afd-0624-46ce-bda0-e2e2f814b0e6/deployments/87e2501b-8386-4b28-b7c3-02b9fa191cef/online'